In [1]:
%history -g -f history.txt

File 'history.txt' exists. Overwrite? y
Overwriting file.


In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import convertdate
import json
from tqdm.notebook import tqdm
from climdist.utils import load_df

In [313]:
main_df = load_df('main')

In [372]:
# (?:Aus |AuS |Vom |Aus dem |(Schreiben aus ))?(?P<placename>(St.\s|Bad\s|San\s|Buenos\s|New\s|[A-ZÜÖÄ][a-züöäSß]{2,}\s?[-–=]\s?|[A-ZÜÖÄ][a-züöäSß]{3,}\s[Aa]m\s)?[A-ZÜÖÄ][a-züöäSß]{3,})(?:\.|,|\.,)\s*(?:([Dd]en)|([Vv]om)|([Dd]em)|([Dd]er»)|([Dd]er))?\s*(?P<date>\d{1,2})(?:\.|ten|sten)?\s*(?P<date2>\(\d{1,2}\.?(\s)?\))?\s+(?P<month>([Jj]an)|([Ff]eb)|([Mm][äa]r)|([Aa]pr)|([Mm]ai)|([Jj][un]n)|([Jj][un]l)|([Aa]ug)|([Ss]ept)|([Oo]ct)|([Nn]ov)|([Dd]e[cz])|(c\.?)|(d\.\sM\.?)|(v\.\sM\.?))

In [4]:
pattern = re.compile('(?P<placename>(([A-Za-züöäSß]{3,}(?=\sGr[eä]nze))\s|St.\s|Bad\s|San\s|Buenos\s|New\s|[A-ZÜÖÄ][a-züöäSß]{2,}\s?[-–=]\s?|[A-ZÜÖÄ][a-züöäSß]{3,}\s[Aa]m\s)?[A-ZÜÖÄ][a-züöäSß]{3,})(?:\.|,|\.,)\s*(?:([Dd]en)|([Vv]om)|([Dd]em)|([Dd]er»)|([Dd]er))?\s*(?P<date>\d{1,2})(?:\.|ten|sten)?\s*(?P<date2>\(\d{1,2}\.?(\s)?\))?\s+(?P<month>([Jj]an)|([Ff]eb)|([Mm][äa]r)|([Aa]pr)|([Mm]ai)|([Jj][un]n)|([Jj][un]l)|([Aa]ug)|([Ss]ept)|([Oo]ct)|([Nn]ov)|([Dd]e[cz])|(c\.?)|(d\.\sM\.?)|(v\.\sM\.?))')


In [222]:
weekdays = ['Montag', 'Montage', 'Dienstag', 'Dienstage', 'Dinstag', 'Mittwoch', 'Donnerstag', 'Donnerstage',
            'DonnerStag', 'DienStag', 'DienStage', 'DinStag',
            'Freitag', 'Freytag', 'Freitage', 'Sonntag', 'Sonnabend', 'Sonntage', 'Stag', 'Tage',
            'Gestern', 'Morgen']
months = ['Januar', 'Februar', 'März', 'April', 'Juni', 'Juny', 'Juli', 'July', 'August',
          'October', 'Oktober', 'November', 'December', 'Dezember']
times_of_day = ['Abend', 'Heute']
currencies = ['Thlr']
other = ['Fremde', 'Hotel', 'Adler', 'Rathhaus', 'Riga-Rathhaus', 'Riga-RathhauS', 'Riga-Schloß', 'Geldkurs', 'Hause',
         'Tgbf', 'Börse', 'Pastorat', 'Senat', 'Schmidt', 'Verwaltung', 'Bekanntmachungen', 'Anzeige', 'Jahre',
         'Tagesbefehle', 'Schloß', 'Ritterhause', 'Weihnachtsabend']

exceptions = weekdays + months + times_of_day + currencies + other

In [264]:
with open('re_exceptions.txt', 'r', encoding='utf8') as f:
    exceptions = []
    for line in f.readlines():
        exceptions.append(line.strip('\n'))

In [265]:
with open('re_pattern.txt', 'r', encoding='utf8') as f:
    pattern = re.compile(f.read())

In [275]:
def get_text(ix, window):
    window_before, window_after = window, window
    if window_before > df.loc[ix, 'start']:
        window_before = df.loc[ix, 'start']
        
    return main_df.loc[df.loc[ix, 'doc_id'], 'full_text'][df.loc[ix, 'start']-window_before:df.loc[ix, 'end']+window_after]
    
    
def get_example(placename, window=200):
    example = df.loc[df.placename == placename].sample(1)
    ix = example.index.values[0]
    doc_id = example.doc_id.values[0]
    print('doc_id: ', doc_id, '\n') 
    if window == 'full':
        print(main_df.loc[doc_id, 'full_text'], '\n')
    else:
        print(get_text(ix, window), '\n')
    for match in re.finditer(pattern, main_df.loc[doc_id, 'full_text']):
        print(match.groupdict())

        
def scan_placenames_dates(main_df):
    
    results = []
    
    for ix, row in tqdm(main_df.iterrows()):
        matches = re.finditer(pattern, row.full_text)
        for m in matches:
            results.append([ix, row.date] + list(m.groupdict().values()) + [m.span()[0], m.span()[1]])
        
    result_df = pd.DataFrame(columns=['doc_id', 'doc_date', 'placename', 'day', 'day2', 'month', 'start', 'end'],
                        data=results).fillna(pd.NA)
    
    result_df['doc_date'] = pd.to_datetime(result_df['doc_date'], format='%Y-%m-%d')
    result_df['year'] = result_df['doc_date'].dt.year
    
    result_df['day'] = result_df['day'].astype(int)
    result_df['day'] = result_df['day'].apply(lambda x: x if (x in range(1,32)) else pd.NA)
    
    result_df['day2'] = result_df['day2'].str.lstrip('( ').str.strip('.) ')
    result_df['day2'].fillna(0, inplace=True)
    result_df['day2'] = result_df['day2'].astype(int)
    result_df.replace(0, pd.NA, inplace=True)

    return result_df[~result_df.placename.isin(exceptions)]

In [276]:
df = scan_placenames_dates(main_df)

0it [00:00, ?it/s]

In [277]:
len(df)

9683

In [36]:
print(df.placename.value_counts().sum())
print(df.placename.value_counts()[lambda x: x >= 30].sum())

9683
6923


In [31]:
placename_replacement_dict = {
    'Petersburg': 'St. Petersburg',
    'Trieft': 'Triest',
    'Pesth': 'Pest',
    'Pefth': 'Pest',
    'Verlin': 'Berlin',
    'Maingegenden': 'Main',
    'Janeiro': 'Rio de Janeiro',
    'Bucharest': 'Bukarest',
    'Vondon': 'London',
    'Loudon': 'London',
    'Rewyork': 'New York',
    'Newyork': 'New York',
    'Ncwyork': 'New York',
    'Newyort': 'New York',
    'New-York': 'New York',
    'Ntwyork': 'New York',
    'Pans': 'Paris',
    'PanS': 'Paris',
    'Parts': 'Paris',
    'PartS': 'Paris',
    'Baris': 'Paris',
    'PariS': 'Paris',
    'Konftantinopel': 'Konstantinopel',
    'Konstautinopel': 'Konstantinopel',
    'Koustantinopel': 'Konstantinopel',
    'Konstantmopel':  'Konstantinopel',
    'Constantinopel': 'Konstantinopel',
    'Kalkutta': 'Calcutta',
    'Niga': 'Riga',
    'Riza': 'Riga',
    'Rlga': 'Riga',
    'Kiga': 'Riga',
    'Rcval': 'Reval',
    'Neval': 'Reval',
    'Torpat': 'Dorpat',
    'DomeSneeS': 'Domesnees',
    'DomesneeS': 'Domesnees',
    'DomeSnees': 'Domesnees',
    'Domcsnees': 'Domesnees',
    'Snees': 'Domesnees',
    'Boldcraa': 'Bolderaa',
    'Sselo': 'Zarskoje Sselo',
    'Skoje-Sselo': 'Zarskoje Sselo',
    'Mayn': 'Main',
    'Liban': 'Libau',
    'Turm': 'Turin',
    'Ancona': 'Ankona',
    'Carlsruhe': 'Karlsruhe',
    'Bafel': 'Basel',
    'Straszburg': 'Straßburg',
    'Strasburg': 'Straßburg',
    'Moskwa': 'Moskau',
    'MoSkau': 'Moskau',
    'Narowa': 'Narwa',
    'Narva': 'Narwa',
    'Berlm': 'Berlin',
    'Bertin': 'Berlin',
    'Berlw': 'Berlin',
    'Verlin': 'Berlin',
    'Mayland': 'Mailand',
    'Darmftadt': 'Darmstadt',
    'DreSden': 'Dresden',
    'Cassel': 'Kassel',
    'Wieu': 'Wien',
    'Manheim': 'Mannheim',
    'Stuttgardt': 'Stuttgart',
    'Frankfurt': 'Frankfurt am Main',
    'TifliS': 'Tiflis',
    'Buenos-Ayres': 'Buenos Aires',
    'Archangel': 'Archangelsk',
    'Kadix': 'Cadix',
    'Malehin': 'Malchin',
    'Türkischen Gränze': 'Türkische Grenze',
    'Türkische Gränze': 'Türkische Grenze',
    'Italienischen Gränze': 'Italienische Grenze',
    'Italienischen Grenze': 'Italienische Grenze',
    'Serbischen Gränze': 'Serbische Grenze',
    'Servische Gränze': 'Serbische Grenze',
    'Polnischen Grenze': 'Polnische Grenze',
    'Spanische Gränze': 'Spanische Grenze',
}

with open('../rz_acta/placename_replacement_dict.json', 'w', encoding='utf8') as f:
    json.dump(placename_replacement_dict, f)

In [278]:
df.placename.replace(placename_replacement_dict, inplace=True)

In [308]:
df[df.placename == 'Sburg']

,doc_id,doc_date,placename,day,day2,month,start,end,year,origin_date,delta
66104,100053,1849-11-25,Sburg,19,<NA>,11,36,50,1849,1849-11-19,6
72060,105290,1851-01-19,Sburg,13,<NA>,1,36,50,1851,1851-01-13,6
72502,105861,1851-03-07,Sburg,2,<NA>,3,36,49,1851,1851-03-02,5
73368,106875,1851-05-24,Sburg,19,<NA>,5,36,50,1851,1851-05-19,5
74376,108219,1851-09-07,Sburg,3,<NA>,9,36,50,1851,1851-09-03,4
128183,153238,1861-09-25,Sburg,22,<NA>,9,6,21,1861,1861-09-22,3
137246,163392,1863-12-14,Sburg,13,<NA>,12,820,834,1863,1863-12-13,1
141242,168235,1864-12-24,Sburg,23,<NA>,12,13537,13551,1864,1864-12-23,1
166546,195196,1870-01-10,Sburg,9,<NA>,1,1163,1176,1870,1870-01-09,1
167011,195909,1870-02-28,Sburg,27,<NA>,2,1962,1976,1870,1870-02-27,1


In [307]:
print(df.placename.value_counts()[lambda x: x > 29].to_string())

Paris                   26165
London                  22561
Berlin                  21787
St. Petersburg          20047
Wien                    15363
Madrid                   5831
Konstantinopel           4628
Riga                     4521
Brüssel                  3487
Kopenhagen               3412
New York                 3342
München                  3266
Hamburg                  2774
Turin                    2580
Dorpat                   1995
Bern                     1890
Stockholm                1696
Dresden                  1695
Reval                    1655
Warschau                 1626
Neapel                   1547
Florenz                  1487
Amsterdam                1445
Haag                     1437
Athen                    1435
Domesnees                1429
Odessa                   1423
Bukarest                 1379
Lissabon                 1345
Hannover                 1318
Pest                     1290
Stuttgart                1285
Triest                   1228
Versailles

In [315]:
get_text(218701, 50)

' do. do. Uem ttiinj, Am 4« 2,5 —\n+^6 0,3\n, Nachts Regen.\n\n\t4. October.\nVaromcter bei 0° C. mm... Lufttemperatur in '

In [311]:
df[df.placename == 'Regen']

,doc_id,doc_date,placename,day,day2,month,start,end,year,origin_date,delta
92019,126229,1855-05-07,Regen,9,<NA>,4,2386,2399,1855,1855-04-09,28
179331,209442,1872-08-30,Regen,16,<NA>,6,11720,11738,1872,1872-06-16,75
217597,253968,1880-07-14,Regen,12,<NA>,7,507,523,1880,1880-07-12,2
217608,253984,1880-07-15,Regen,13,<NA>,7,521,537,1880,1880-07-13,2
217708,254106,1880-07-23,Regen,22,<NA>,7,483,499,1880,1880-07-22,1
218283,254827,1880-09-06,Regen,5,<NA>,9,608,624,1880,1880-09-05,1
218460,255035,1880-09-19,Regen,18,<NA>,9,600,617,1880,1880-09-18,1
218701,255314,1880-10-06,Regen,4,<NA>,10,493,508,1880,1880-10-04,2
220400,257675,1881-02-26,Regen,25,<NA>,2,636,652,1881,1881-02-25,1
221444,259151,1881-05-30,Regen,29,<NA>,5,513,529,1881,1881-05-29,1


In [38]:
def cleanup_dates(df):
    
    #df = df.copy()
    
    month_dict = dict(zip(['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun',
                           'Jul', 'Aug', 'Sept', 'Okt', 'Nov', 'Dec'], range(1,13)))
    
    df.month = df.month.str.capitalize()
    df.month.replace({'Dez': 'Dec', 'Mar': 'Mär', 'Oct': 'Okt', '0ct': 'Okt', '0kt': 'Okt',
                      'Jnl': 'Jul', 'Jnn': 'Jun', 'Juu': 'Jun', 'May': 'Mai'}, inplace=True)
    
    df.month.replace(month_dict, inplace=True)
    df.loc[df.month.isin(['C', 'C.', 'D. m', 'D. m.']), 'month'] = df.loc[df.month.isin(['C', 'C.', 'D. m', 'D. m.']), 'doc_date'].dt.month
    df.loc[df.month.isin(['V. m', 'V. m.']), 'month'] = (df.loc[df.month.isin(['v. M', 'V. m.']), 'doc_date'] - pd.DateOffset(months=1)).dt.month - 1
    
    df.month = df.month.fillna(0).astype(int)
    df.month = df.month.apply(lambda x: x if (x in range(1,13)) else pd.NA)
    
    df.day = df.day.fillna(0).astype(int)
    df.day = df.day.apply(lambda x: x if (x in range(1,32)) else pd.NA)
    
    df.day2 = df.day2.fillna(0).astype(int)
    df.day2 = df.day2.apply(lambda x: x if (x in range(1,32)) else pd.NA)
    
    return df 

In [53]:
df = cleanup_dates(df)

In [46]:
def verify_dates(ix, df=df):
    
    #print(df.loc[ix,['day', 'day2', 'month', 'doc_date']])
    
    day, day2, month, doc_date = df.loc[ix,['day', 'day2', 'month', 'doc_date']]
    
    if type(day) == pd._libs.missing.NAType or type(month) == pd._libs.missing.NAType:
        return pd.NA
    
    # 2 kp
    if type(day2) == int:
        #print('difference: ', max(day, day2) - min(day, day2))
        if month == doc_date.month or month < doc_date.month:
            if day > day2 and day > doc_date.day:
                # julian is the smaller one
                origin_date = pd.to_datetime(f'{str(doc_date.year)}-{month}-{day2}',
                                             format='%Y-%m-%d', errors='coerce')
            else: # kontrolli see üle
                origin_date = pd.to_datetime(f'{str(doc_date.year)}-{month}-{day}',
                                             format='%Y-%m-%d', errors='coerce')
                
        elif month > doc_date.month:
            if month == 12 and doc_date.month == 1:
                if day > day2 and day > doc_date.day:
                    origin_date = pd.to_datetime(f'{str(doc_date.year-1)}-{month}-{day2}',
                                                 format='%Y-%m-%d', errors='coerce')
                else:
                    origin_date = pd.to_datetime(f'{str(doc_date.year-1)}-{month}-{day}',
                                             format='%Y-%m-%d', errors='coerce')
                    
            else:
                return pd.NA
                
            
    
    # 1 kp
    else:
        # mis on kuu võrreldes lehe kuuga?
        if month < doc_date.month:
            # origin is julian, apply directly to datetime
            origin_date = pd.to_datetime(f'{str(doc_date.year)}-{month}-{day}',
                                         format='%Y-%m-%d', errors='coerce')
        
        elif month == doc_date.month:
            if day > doc_date.day:
                # origin is gregorian, convert to julian first, then apply
                try:
                    day_jul = convertdate.julian.from_gregorian(year=doc_date.year, month=month, day=day)
                except ValueError:
                    return pd.NA
                origin_date = pd.to_datetime(f'{str(day_jul[0])}-{str(day_jul[1])}-{str(day_jul[2])}',
                                             format='%Y-%m-%d', errors='coerce')
            else:
                # origin is julian, apply directly to datetime
                origin_date = pd.to_datetime(f'{str(doc_date.year)}-{month}-{day}',
                                             format='%Y-%m-%d', errors='coerce')
        
        elif month > doc_date.month:
            if month == 12 and doc_date.month == 1:
                # origin is from last year - probably in julian b/c is smaller that doc_date and there is no day2
                origin_date = pd.to_datetime(f'{str(doc_date.year-1)}-{month}-{day}',
                                             format='%Y-%m-%d', errors='coerce')
            else:
                # origin is from next month, thus in gregorian
                try:
                    day_jul = convertdate.julian.from_gregorian(year=doc_date.year, month=month, day=day)
                except ValueError:
                    return pd.NA
                origin_date = pd.to_datetime(f'{str(day_jul[0])}-{str(day_jul[1])}-{str(day_jul[2])}',
                                             format='%Y-%m-%d', errors='coerce')
            
    return origin_date

In [54]:
origin_dates = []
for ix in tqdm(df.index):
    origin_dates.append(verify_dates(ix, df))

df['origin_date'] = origin_dates

  0%|          | 0/279436 [00:00<?, ?it/s]

In [184]:
df.dropna(subset=['origin_date'], inplace=True)
df['origin_date'] = pd.to_datetime(df.origin_date)

In [189]:
df.origin_date

0        1801-12-20
1        1801-12-18
2        1801-12-22
3        1801-12-16
4        1801-12-19
            ...    
299594   1888-12-27
299607   1888-12-28
299608   1888-12-24
299610   1888-12-29
299612   1888-12-29
Name: origin_date, Length: 243572, dtype: datetime64[ns]

In [193]:
test_df = pd.read_parquet('../data/processed/RZ_sample.parquet')

In [194]:
test_df

,date,year,month,day,pub,heading,full_text,href,text_len,placename,origin_date,origin_month,heading2
158691,1862-12-12,1862,12,12,Rigasche Zeitung,Holland.,"Holland.\n\n\tHaag, 13. (1.) December. sKöln.Z...",https://proc.dom.lndb.lv/file.axd?id=15287209&...,696,None,None,None,Holland
120911,1854-04-05,1854,4,5,Rigasche Zeitung,Inländische Nachrichten.,Inländische Nachrichten.\n\n\tTt. Petersburg. ...,https://proc.dom.lndb.lv/file.axd?id=12135834&...,7271,None,None,None,Inländische Nachrichten
218663,1874-07-13,1874,7,13,Rigasche Zeitung,Das Attentat und die Reaction.,Das Attentat und die Reaction.\n\n\tIn dem Kam...,https://proc.dom.lndb.lv/file.axd?id=11835060&...,9780,None,None,None,Das Attentat und die Reaction
158839,1862-12-22,1862,12,22,Rigasche Zeitung,Oesterreich.,"Oesterreich.\n\n\tWien, 27. (15.) December. De...",https://proc.dom.lndb.lv/file.axd?id=4374749&a...,2573,None,None,None,Oesterreich
204682,1871-10-13,1871,10,13,Rigasche Zeitung,Benedetti und Lessinnes,"Benedetti und Lessinnes\n\n\tL. Berlin, 22. (1...",https://proc.dom.lndb.lv/file.axd?id=13290475&...,5345,None,None,None,Benedetti und Lessinnes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66739,1841-09-02,1841,9,2,Rigasche Zeitung,"Paris, den 3. September.","Paris, den 3. September.\n\n\tIm „Moniteur par...",https://proc.dom.lndb.lv/file.axd?id=8950120&a...,4589,Paris,3,Sept,Paris
4746,1806-01-20,1806,1,20,Rigasche Zeitung,"Folgende Personen sind gesonnen, bon hier Zu r...","Folgende Personen sind gesonnen, bon hier Zu r...",https://proc.dom.lndb.lv/file.axd?id=8565650&a...,717,None,None,None,Folgende Personen sind gesonnen von hier zu re...
10539,1810-03-26,1810,3,26,Rigasche Zeitung,"Madrid, den 28. Februar.","Madrid, den 28. Februar.\n\n\tUnter den Zügen,...",https://proc.dom.lndb.lv/file.axd?id=15202948&...,1403,Madrid,28,Feb,Madrid
119315,1853-12-19,1853,12,19,Rigasche Zeitung,Inländische Nachrichten.,Inländische Nachrichten.\n\n\n\tSt. Petersburg...,https://proc.dom.lndb.lv/file.axd?id=4090652&a...,2616,None,None,None,Inländische Nachrichten


In [195]:
scan_placenames_dates(test_df, exceptions)

0it [00:00, ?it/s]

,doc_id,doc_date,placename,day,day2,month,start,end,year
0,158691,1862-12-12,Haag,13,1,Dec,11,29,1862
1,158839,1862-12-22,Wien,27,15,Dec,15,34,1862
2,204682,1871-10-13,Berlin,22,10,Oct,29,50,1871
3,11239,1810-08-06,Paris,27,<NA>,Jul,1569,1590,1810
4,130806,1856-06-26,Kopenhagen,2,<NA>,Jul,12,30,1856
...,...,...,...,...,...,...,...,...,...
10379,110045,1852-01-26,Brüssel,31,<NA>,Jan,0,16,1852
10380,64985,1841-01-04,Konstantinopel,23,<NA>,Dec,0,27,1841
10381,66739,1841-09-02,Paris,3,<NA>,Sept,0,18,1841
10382,10539,1810-03-26,Madrid,28,<NA>,Feb,0,19,1810


In [201]:
pattern2

re.compile(r'(?P<placename>(([A-Za-züöäSß]{3,}(?=\sGr[eä]nze))\s|St.\s|Bad\s|San\s|Buenos\s|New\s|[A-ZÜÖÄ][a-züöäSß]{2,}\s?[-–=]\s?|[A-ZÜÖÄ][a-züöäSß]{3,}\s[Aa]m\s)?[A-ZÜÖÄ][a-züöäSß]{3,})(?:\.|,|\.,)\s*(?:([Dd]en)|([Vv]om)|([Dd]em)|([Dd]er»)|([Dd]er))?\s*(?P<date>\d{1,2})(?:\.|ten|sten)?\s*(?P<date2>\(\d{1,2}\.?(\s)?\))?\s+(?P<month>([Jj]an)|([Ff]eb)|([Mm][äa]r)|([Aa]pr)|([Mm]ai)|([Jj][un]n)|([Jj][un]l)|([Aa]ug)|([Ss]ept)|([Oo]ct)|([Nn]ov)|([Dd]e[cz])|(c\.?)|(d\.\sM\.?)|(v\.\sM\.?))\n',
re.UNICODE)

In [204]:
pattern2

re.compile(r'(?P<placename>(([A-Za-züöäSß]{3,}(?=\sGr[eä]nze))\s|St.\s|Bad\s|San\s|Buenos\s|New\s|[A-ZÜÖÄ][a-züöäSß]{2,}\s?[-–=]\s?|[A-ZÜÖÄ][a-züöäSß]{3,}\s[Aa]m\s)?[A-ZÜÖÄ][a-züöäSß]{3,})(?:\.|,|\.,)\s*(?:([Dd]en)|([Vv]om)|([Dd]em)|([Dd]er»)|([Dd]er))?\s*(?P<date>\d{1,2})(?:\.|ten|sten)?\s*(?P<date2>\(\d{1,2}\.?(\s)?\))?\s+(?P<month>([Jj]an)|([Ff]eb)|([Mm][äa]r)|([Aa]pr)|([Mm]ai)|([Jj][un]n)|([Jj][un]l)|([Aa]ug)|([Ss]ept)|([Oo]ct)|([Nn]ov)|([Dd]e[cz])|(c\.?)|(d\.\sM\.?)|(v\.\sM\.?))\n',
re.UNICODE)

In [205]:
with open('placename_replacement_dict.json', 'r', encoding='utf8') as f:
    placename_replacement_dict = json.load(f)

In [281]:
df = pd.read_csv('processed_data.tsv', sep='\t', encoding='utf8').convert_dtypes()

In [282]:
df['doc_date'] = pd.to_datetime(df['doc_date'])

In [283]:
df['origin_date'] = pd.to_datetime(df['origin_date'])

In [284]:
df[df.placename == 'Paris']

,doc_id,doc_date,placename,day,day2,month,start,end,year,origin_date,delta
1,1,1802-01-01,Paris,18,<NA>,12,<NA>,20,1802,1801-12-18,14
17,20,1802-01-08,Paris,25,<NA>,12,<NA>,21,1802,1801-12-25,14
32,45,1802-01-15,Paris,1,<NA>,1,<NA>,20,1802,1802-01-01,14
35,53,1802-01-18,Paris,6,<NA>,1,<NA>,19,1802,1802-01-06,12
47,73,1802-01-25,Paris,8,<NA>,1,<NA>,19,1802,1802-01-08,17
...,...,...,...,...,...,...,...,...,...,...,...
243508,289479,1888-12-14,Paris,28,11,12,<NA>,20,1888,1888-12-11,3
243522,289519,1888-12-16,Paris,27,15,12,1532,1552,1888,1888-12-15,1
243527,289557,1888-12-19,Paris,29,17,12,<NA>,20,1888,1888-12-17,2
243531,289558,1888-12-19,Paris,30,13,12,1880,1900,1888,1888-12-13,6


243569

In [260]:
placename_replacement_dict

{'Petersburg': 'St. Petersburg',
 'Trieft': 'Triest',
 'Pesth': 'Pest',
 'Pefth': 'Pest',
 'Verlin': 'Berlin',
 'Maingegenden': 'Main',
 'Janeiro': 'Rio de Janeiro',
 'Bucharest': 'Bukarest',
 'Vondon': 'London',
 'Loudon': 'London',
 'Rewyork': 'New York',
 'Newyork': 'New York',
 'Ncwyork': 'New York',
 'Newyort': 'New York',
 'New-York': 'New York',
 'Ntwyork': 'New York',
 'Pans': 'Paris',
 'PanS': 'Paris',
 'Parts': 'Paris',
 'PartS': 'Paris',
 'Baris': 'Paris',
 'PariS': 'Paris',
 'Konftantinopel': 'Konstantinopel',
 'Konstautinopel': 'Konstantinopel',
 'Koustantinopel': 'Konstantinopel',
 'Konstantmopel': 'Konstantinopel',
 'Constantinopel': 'Konstantinopel',
 'Kalkutta': 'Calcutta',
 'Niga': 'Riga',
 'Riza': 'Riga',
 'Rlga': 'Riga',
 'Kiga': 'Riga',
 'Rcval': 'Reval',
 'Neval': 'Reval',
 'Torpat': 'Dorpat',
 'DomeSneeS': 'Domesnees',
 'DomesneeS': 'Domesnees',
 'DomeSnees': 'Domesnees',
 'Domcsnees': 'Domesnees',
 'Snees': 'Domesnees',
 'Boldcraa': 'Bolderaa',
 'Sselo': 'Zarsko